In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
#import json
import os

import pandas as pd
pd.set_option("display.max_columns", 30)
import requests

In [ ]:
# Extract T - 2 months' data
%run utilities/U01_current_formated_datatime.ipynb

#url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp>='{formated_datetime}T00:00:00' AND trip_start_timestamp<='{formated_datetime}T23:59:59'&$limit=300"
url = "https://data.cityofchicago.org/resource/ajtu-isnz.json"

params = f"$where=trip_start_timestamp>='{formated_datetime}T00:00:00' AND trip_start_timestamp<='{formated_datetime}T23:59:59'&$limit=30000"

headers = {"X-App-Token": os.environ.get("CHICAGO_API_TOKEN")}

response = requests.get(url, headers = headers, params = params)

response

In [ ]:
data = response.json()
len(data)

In [ ]:
taxi_trips = pd.DataFrame(data)
taxi_trips.head()

In [ ]:
#taxi_trips.head()
taxi_trips.info()

In [ ]:

taxi_trips.describe()

### Transformation: deal with NaN values

In [ ]:
taxi_trips[taxi_trips["fare"].isna()].sample(5)

In [ ]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

In [ ]:
taxi_trips.dropna(inplace=True)

In [ ]:
taxi_trips.info()

### Transformation: renaming

In [ ]:
taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                           "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

In [ ]:
taxi_trips.columns

### Transformation: create helper column for weather

In [ ]:
taxi_trips["trip_start_timestamp"] = pd.to_datetime(taxi_trips["trip_start_timestamp"])
taxi_trips["trip_end_timestamp"] = pd.to_datetime(taxi_trips["trip_end_timestamp"])

In [ ]:
taxi_trips["datetime_for_weather"] = taxi_trips["trip_start_timestamp"].dt.floor("H")

In [ ]:
taxi_trips.info()

### Check joining the trips and the weather data

In [ ]:
# Get weather data: Extract part

url = "https://archive-api.open-meteo.com/v1/era5"

params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formated_datetime,
    "end_date": formated_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}

response = requests.get(url, params = params)

weather_data = response.json()

weather_data["hourly"]

In [ ]:
# Get weather data: Transform part

weather_df = pd.DataFrame(weather_data['hourly'])

weather_df['time'] = pd.to_datetime(weather_df['time'])

# translate headers based on dictionary
header_dict = {'time': 'datetime',
    'temperature_2m': 'temperature',
    'wind_speed_10m': 'wind_speed'}
weather_df.columns = pd.Series(weather_df.columns).replace(header_dict)

weather_df

In [ ]:
taxi_trips_with_weather = taxi_trips.merge(weather_df, left_on="datetime_for_weather", right_on="datetime")

In [ ]:
taxi_trips_with_weather.head()

### Transformation: Data type conversions

In [ ]:
taxi_trips.info()

In [ ]:
taxi_trips.head(1)

In [ ]:
data_types = {
    "trip_start_timestamp": "datetime64[ns]",
    "trip_end_timestamp": "datetime64[ns]",
    "trip_seconds": "int32",
    "trip_miles": "float",
    "pickup_community_area_id": "int8",
    "dropoff_community_area_id": "int8",
    "fare": "float",
    "tips": "float",
    "tolls": "float",
    "extras": "float",
    "trip_total": "float",
    "pickup_centroid_latitude": "float",
    "pickup_centroid_longitude": "float",
    "dropoff_centroid_latitude": "float",
    "dropoff_centroid_longitude": "float"
}

taxi_trips = taxi_trips.astype(data_types)

### Memory usage

In [ ]:
taxi_trips.memory_usage(deep=True).sum()